<a href="https://colab.research.google.com/github/Just1got/Hello_python/blob/main/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8_%D0%B0%D0%BA%D1%82%D0%B8%D0%B2%D0%B0%D1%86%D0%B8%D0%B8_v2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Функции активации



Сегодня обсудим:
1. Какие есть функции активации нейронов (Sigmoid, ReLU, ELU, Leaky ReLU...)
2. Плюсы каждой функции активации
3. Минусы каждой функции активации
4. Как написать свою функцию активации

## Для чего нужны?

<img src='https://drive.google.com/uc?id=1gF12si7S9po9bBfJ9Q7bfmM5v55vi3tE'>

Наш нейрон из себя представляет взвешенную сумму:

$$y = \sum{XW} + bias$$

И значение $y$ может изменяться от $-∞$ до $+∞$. И значит *всегда* этот нейрон будет пропускать через себя сигнал. У него **нет границы** активации.

А если вспомним биологический нейрон, то чтобы нейрон активировался (пропустил информацию дальше), нужно поднакопить необходимый сигнал. А значит, раз искусственный нейрон основан на биологическом нейроне, то и **активировать** его мы будем должным образом.

И поможет нам в этом фукнция активации. Ее задача заключается в том, чтобы проверять, является ли нейрон активированным, или его можно проигнорировать.

## Основная задача на сегодня

In [ ]:
from keras.datasets import fashion_mnist

(X_train, y_train_labels), (X_test, y_test_labels) = fashion_mnist.load_data()

In [ ]:
class_names = ['T-shirt/top',
               'Trouser',
               'Pullover',
               'Dress',
               'Coat',
               'Sandal',
               'Shirt',
               'Sneaker',
               'Bag',
               'Ankle boot']

In [ ]:
y_train_labels

In [ ]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train_labels)
y_test = to_categorical(y_test_labels)

In [ ]:
y_train

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(X_train[0], cmap='gray')
plt.colorbar()
plt.show()

Нормализуем данные, из диапазона от 0 до 255 переводим в диапазон от 0 до 1.

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

### Линейная функция

$$A(x) = x$$

Линейная функция представляет собой прямую линию и она пропорциональна входу.

In [ ]:
from keras.activations import linear
import matplotlib.pyplot as plt
import numpy as np


x = np.linspace(-10, 10, 100)
y = linear(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.ylabel('output')
plt.xlabel('input')
plt.title('linear')
plt.legend()
plt.grid();


Производная такой функции по $x$ равняется:

$$\frac{∂A}{∂x} = 1$$

In [ ]:
x = np.linspace(-10, 10, 100)
y = linear(x)

grad_y = np.ones_like(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.plot(x, grad_y, c='g', label='grad')
plt.ylabel('output')
plt.xlabel('input')
plt.title('linear')
plt.legend()
plt.grid();

#### Первая проблема

Для этой функции производная всегда одинакова в любой точке.

А это значит, что градиент никак не связан с входом ($x$). Градиент - постоянный вектор, а спуск идет по **постоянному** градиенту. Если есть ошибочное предсказание, то изменения, сделанные обратным распространением ошибки, тоже постоянны вне зависимости от входа.

Чтобы это проверить, создадим обучающие данные, будем обучать нейронную сеть на воспроизведение входа.

In [ ]:
import numpy as np

X = np.array([[1], [3], [2]])
y = np.array([[1, 3, 2]]).T

Сделаем один линейные слой с линейной активацией и уберем свободный вес.

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
import tensorflow as tf
tf.random.set_seed(1)


model = Sequential([
    Dense(1, input_shape=(1,), activation='linear', use_bias=False),
])

model.summary()

In [ ]:
model.get_weights()

Будем следить за градиентами на очень низком уровне, для этого нам потребуется объект из tensorflow `GradientTape`, который записывает градиенты.

In [ ]:
from keras.losses import MSE


for i in range(3):     
    with tf.GradientTape() as tape: 
        pred = model(X[i])
        print('Prediction is', pred.numpy(), 'True is', y[i])

        loss_value = MSE(y[i], pred)
        print('Loss is', loss_value.numpy())

        grads = tape.gradient(loss_value, model.trainable_weights) 
        print('Grad are', grads / loss_value)
        print('_' * 40)

#### Вторая проблема


<img src='https://drive.google.com/uc?id=1pLd9DW7HxfcxoXyS3mRaENS8ugpxeSou' width=800>


Раз каждый слой активируется линейной функцией, то в следующий слой пойдет линейная связь от входа. В другом слое получаем линейную комбинацию, прогоняем через линейную активацию и дальше.

Нет разницы, сколько у нас слоев. Ведь финальная функция активации в последнем слое будет просто линейной функцией от входных параметров на первом слое.

Значит, любое количество слоев можно заменить одним. Больше нет смысла ставить подряд слои с линейной активацией.



### Сигмоида

Сигмоида - одна из самых частых активационных функций в нейросетях.


$$A(x) = \frac{1}{1 + e^{-x}}$$


**Преимущества:**
1. Сигмоида — **нелинейная функция**, а комбинация таких функций производит тоже нелинейную функцию. А значит проблема линейной функция здесь неактуальна, мы можем обучать подряд **много слоев** и это будет иметь смысл.
2. В диапазоне значений сигнала от -2.5 до 2.5 значения активации меняются очень быстро. А значит, что любое изменение значения сигнала в этой области повлечет существенное изменение значения активации. 
3. Сигмоида идеально подходит для задач **бинарной классификации**. Она стремится привести значения к одной из сторон кривой, по итогу получаем четкие границы предсказания
4. У сигмоиды фиксированный диапазон значений функции — [0,1], а линейная функция обладает диапазоном ($-∞$, $+∞$). А значит в случае с сигмоидой **не будет ошибок** с большими значениями активаций




In [ ]:
from keras.activations import sigmoid


x = np.linspace(-10, 10, 100)
y = sigmoid(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.ylabel('output')
plt.xlabel('input')
plt.title('sigmoid')
plt.legend()
plt.grid();

Производная сигмоиды

$$\sigma(z) = \frac{1}{1 + exp(-z)}$$

$$\frac{d\sigma(z)}{dz} = -\frac{1}{(1 + exp(-z))^2}(-exp(-z)) = \frac{exp(-z)}{(1 + exp(-z))^2} \text{(*)} = \frac{1}{1 + exp(-z)}(1-\frac{1}{1 + exp(-z)}) = \sigma(1-\sigma)$$

$ (*)\frac{exp(-z) +1-1}{(1 + exp(-z))^2} = \frac{1 + exp(-z) -1}{(1 + exp(-z))^2}= \frac{1 + exp(-z)}{(1 + exp(-z))^2} - \frac{1}{(1 + exp(-z))^2}=$

$ = \frac{1}{(1 + exp(-z))} - \frac{1}{(1 + exp(-z))}\frac{1}{(1 + exp(-z))} = \frac{1}{(1 + exp(-z))}(1- \frac{1}{(1 + exp(-z))}) $

In [ ]:
def sigmoid_grad(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [ ]:
x = np.linspace(-10, 10, 100)
y = sigmoid(x)

grad_y = sigmoid_grad(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.plot(x, grad_y, c='g', label='grad')
plt.ylabel('output')
plt.xlabel('input')
plt.title('sigmoid')
plt.legend()
plt.grid();

In [ ]:
0.25 ** 5 * 0.001

#### Проблема

При приближении к концам сигмоиды значения активации очень слабо изменяются. Это означает, что градиент в таких областях имеет очень мальнькие значения. А из-за этого рождается проблема **затухающих градиентов**

Градиент получается настолько маленьким или же вообще нулевым, что обучение замораживается или же идет очень медленно.

In [ ]:
from keras.layers import Flatten
tf.random.set_seed(9)

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='sigmoid'),
    Dense(64, activation='sigmoid'),
    Dense(32, activation='sigmoid'),
    Dense(16, activation='sigmoid'),
    Dense(10, activation='sigmoid')
])


model.summary()

In [ ]:
from keras.losses import CategoricalCrossentropy
#from keras.optimizers.legacy import optimizer_v2
#from keras.optimizer_v2 import gradient_descent

optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=0.01)
loss_fn = CategoricalCrossentropy()

In [ ]:
# Подготовим тренировочный датасет
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)) 
train_dataset = train_dataset.shuffle(buffer_size=X_train.shape[0]).batch(batch_size) 

In [ ]:
for X, y in train_dataset:
    print(X.shape)
    break

In [ ]:
grad_log = []

epochs = 2 

for epoch in range(epochs): 
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset): 
        
        with tf.GradientTape() as tape: 
            preds = model(x_batch_train) 

            loss_value = loss_fn(y_batch_train, preds)

            grads = tape.gradient(loss_value, model.trainable_weights) 
            g_g = []

            for g_s in grads:
                if len(g_s.numpy().shape) == 2:
                    g_g.append(g_s.numpy()[0, 0]) 

        grad_log.append(g_g)

        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 

        if step % 200 == 0:
            print(f'Epoch {epoch + 1}/{epochs}', end='. ')
            print(f'Step {step}. Train loss: {loss_value}') 

In [ ]:
grad_log = np.array(grad_log)

plt.figure(figsize=(16, 5))
plt.title('Grad by layer')
plt.xlabel('№ layer')
plt.ylabel('grad')
plt.grid()
plt.plot(np.abs(grad_log[0, :]), label='step 0')
plt.plot(np.abs(grad_log[10, :]), label='step 10')
plt.plot(np.abs(grad_log[100, :]), label='step 100')
plt.plot(np.abs(grad_log[700, :]), label='step 700')
plt.legend()
plt.show()

### Тангенс

$$tanh(x)={\frac {e^{x}-e^{-x}}{e^{x}+e^{-x}}}={\frac {e^{2x}-1}{e^{2x}+1}} = 2σ(2x) - 1$$

In [ ]:
from keras.activations import tanh


x = np.linspace(-10, 10, 100)
y = tanh(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.ylabel('output')
plt.xlabel('input')
plt.title('tanh')
plt.legend()
plt.grid();

Производная тангенса

$$∂\tanh(z) = \frac{1}{ch^2(x)} = \frac{1}{(\frac{e^x + e^{-x}}{2})^2}$$

$$ch(x) = \frac{e^x + e^{-x}}{2}$$

In [ ]:
def tanh_grad(x):
    return 1 / (np.exp(x) + np.exp(-x) / 2) ** 2

In [ ]:
x = np.linspace(-10, 10, 100)
y = tanh(x)

grad_y = tanh_grad(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.plot(x, grad_y, c='g', label='grad')
plt.ylabel('output')
plt.xlabel('input')
plt.title('tanh')
plt.legend()
plt.grid();

Гиперболический тангенс очень похож на сигмоиду, только вот производная у него чуть больше, но все остальные плюсы и минусы остаются.

**Плюсы:**
1. Тангенс - **нелинейная функция**, можем обучать подряд много слоев и это будет иметь смысл.
2. В диапазоне значений сигнала от -2.5 до 2.5 значения активации меняются очень быстро. А значит, что любое изменение значения сигнала в этой области повлечет существенное изменение значения активации.
3. Тангенс идеально подходит для задач **бинарной классификации**. Он стремится привести значения к одной из сторон кривой.
4. У тангенса фиксированный диапазон значений функции — [-1,1]. А значит не будет ошибок с большими значениями аткиваций


**Минусы:**
1. При приближении к концам тангенса значения активации очень слабо изменяются. Имеем проблему **затухающих градиентов**.

### ReLU


$$A(x) = max(0, x)$$

Может показаться, что у ReLU все те же самые проблемы, что и у линейной функция. Но на самом деле, ReLU нелинейна. А это значит, что мы можем создавать подряд много слоев. 

ReLU менее требовательна к вычислительным ресурсам, чем тангенс или сигмоида, поэтому ReLU пользуется популярностью при создании глубоких нейронных сетей.

In [ ]:
from keras.activations import relu


x = np.linspace(-10, 10, 100)
y = relu(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.ylabel('output')
plt.xlabel('input')
plt.title('ReLU')
plt.legend()
plt.grid();

Производная ReLU

In [ ]:
def relu_grad(x):
    return np.where(x >= 0, 1, 0)

In [ ]:
x = np.linspace(-10, 10, 100)
y = relu(x)

grad_y = relu_grad(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.plot(x, grad_y, c='g', label='grad')
plt.ylabel('output')
plt.xlabel('input')
plt.title('relu')
plt.legend()
plt.grid();

Проверим, что проблемы затухающих градиентов больше нет.

In [ ]:
tf.random.set_seed(9)

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(10, activation='sigmoid')
])


model.summary()

In [ ]:
optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=0.01)
loss_fn = CategoricalCrossentropy()

In [ ]:
grad_log = []

epochs = 2 

for epoch in range(epochs): 
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset): 
        
        with tf.GradientTape() as tape: 
            preds = model(x_batch_train) 

            loss_value = loss_fn(y_batch_train, preds)

            grads = tape.gradient(loss_value, model.trainable_weights) 
            g_g = []

            for g_s in grads:
                if len(g_s.numpy().shape) == 2:
                    g_g.append(g_s.numpy()[0, 0]) 

        grad_log.append(g_g)

        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 

        if step % 200 == 0:
            print(f'Epoch {epoch + 1}/{epochs}', end='. ')
            print(f'Step {step}. Train loss: {loss_value}') 

In [ ]:
grad_log = np.array(grad_log)

plt.figure(figsize=(16, 5))
plt.title('Grad by layer')
plt.xlabel('№ layer')
plt.ylabel('grad')
plt.grid()
plt.plot(np.abs(grad_log[0, :]), label='step 0')
plt.plot(np.abs(grad_log[10, :]), label='step 10')
plt.plot(np.abs(grad_log[100, :]), label='step 100')
plt.plot(np.abs(grad_log[700, :]), label='step 700')
plt.legend()
plt.show()

#### Проблема


Если имеем большую нейронную сеть с огромным количеством нейронов, то использование сигмоиды или гиперболического тангенса повлекут за собой **активацию буквально всех нейронов**. А в идеале хочется, чтобы некоторые нейроны не были активированы, чтобы сеть была легче.

ReLu может это сделать. К примеру, если у нас есть нейросеть со случайно инициализированными весами, то 50% активаций равны 0 по ReLU.

Но и тут есть проблема. Градиент на этой части равен 0. А значит, веса сети не будут изменяться во время обучения.


Но существуют модификации ReLU, которые помогают эту проблему решить.



In [ ]:
X = np.array([[1], [3], [2]])
y = np.array([[1, 3, 2]]).T

Сделаем один линейные слой с активацией ReLU и уберем свободный вес.

In [ ]:
tf.random.set_seed(1)


model = Sequential([
    Dense(1, input_shape=(1,), activation='relu', use_bias=False),
])

model.summary()

In [ ]:
model.get_weights()

Будем следить за градиентами на очень низком уровне.

In [ ]:
for i in range(3):     
    with tf.GradientTape() as tape: 
        pred = model(X[i])
        print('Prediction is', pred.numpy(), 'True is', y[i])

        loss_value = MSE(y[i], pred)
        print('Loss is', loss_value.numpy())

        grads = tape.gradient(loss_value, model.trainable_weights) 
        print('Grad are', grads / loss_value)
        print('_' * 40)

### Leaky ReLU


$$A(x) = max(0.1x, x)$$


Например, для решения проблемы нулевого градиента имеет смысл заменить горизонтальную часть функции на линейную.

Получаем слегка отклоненную линию от горизонтального положения. Суть в том, чтобы сделать градиент неравным нулю и постепенно восстанавливать его во время тренировки.

In [ ]:
from keras.activations import leaky_relu


x = np.linspace(-10, 10, 100)
y = leaky_relu(x, alpha=0.1)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.ylabel('output')
plt.xlabel('input')
plt.title('leaky ReLU')
plt.legend()
plt.grid();

Производная leaky ReLU

In [ ]:
def leaky_relu_grad(x):
    return np.where(x >= 0, 1, 0.1)

In [ ]:
x = np.linspace(-10, 10, 100)
y = leaky_relu(x, alpha=0.1)

grad_y = leaky_relu_grad(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.plot(x, grad_y, c='g', label='grad')
plt.ylabel('output')
plt.xlabel('input')
plt.title('leaky ReLU')
plt.legend()
plt.grid();

### ELU


<img src='https://drive.google.com/uc?id=1bCygDVQiwQP7lu-m0gyQy6EjyCSa1YA9' width=200>

Или же можем нелинейно отклонять линию от горизонтального положения, а более плавно.

Но суть остается такая же - делаем градиент ненулевым.

In [ ]:
from keras.activations import elu


x = np.linspace(-10, 10, 100)
alpha = 0.3
y = elu(x, alpha=alpha)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.ylabel('output')
plt.xlabel('input')
plt.title('elu')
plt.legend()
plt.grid();

Производная ELU

In [ ]:
def elu_grad(x):
    return np.where(x >= 0, 1, elu(x, alpha=alpha) + alpha)

In [ ]:
x = np.linspace(-10, 10, 100)
y = elu(x, alpha=alpha)

grad_y = elu_grad(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y, c='b', label='activation')
plt.plot(x, grad_y, c='g', label='grad')
plt.ylabel('output')
plt.xlabel('input')
plt.title('ELU')
plt.legend()
plt.grid();

### Как выбрать функцию активации?

Как и во всем глубоком обучении, здесь нет очевидного ответа. Выбирайте ту функцию, которая ведет к более быстрому обучению.

Есть только однозначный ответ касаемо функции активации на последнем слое. Если у вас задача классификации, то самый лучший кандидат - sigmoid, т.к. выдает по сути вероятность быть одним классом или другим.

А если задача регрессии, то выбирайте ReLU - если у вас не может быть отрицательного целевого значения или же линейную активацию.

Ну и конечно же, можно использоваться кастомные функции активации.

### Кастомная функция активации

Посмотрим на проблемы очень больших градиентов.

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(10, activation='sigmoid')
])


model.summary()

Для этого поставим очень большие веса.

Такие веса могут получится, если градиент будет всегда большой.

In [ ]:
np.random.seed(9)

new_weights = []
#пройдемся по текущим весам get_weights()
for weights in model.get_weights():
    if len(weights.shape) == 2:
        new_weights.append(np.random.randn(weights.shape[0], weights.shape[1]) * 100) #будет добавлять случайные веса из норм. распред. нужной размерности •100
    else:
        new_weights.append(np.random.randn(weights.shape[0]) * 100) #bias (1 размерноть) – свобоные веса

model.set_weights(new_weights)

model.get_weights()[:2]

In [ ]:
optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=0.01)
loss_fn = CategoricalCrossentropy()

А видим, что уже ошибка посчитаться не может, что-то пошло не так.

In [ ]:
grad_log = []

epochs = 2 

for epoch in range(epochs): 
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset): 
        
        with tf.GradientTape() as tape: 
            preds = model(x_batch_train) 

            loss_value = loss_fn(y_batch_train, preds)

            grads = tape.gradient(loss_value, model.trainable_weights) 
            g_g = []

            for g_s in grads:
                if len(g_s.numpy().shape) == 2:
                    g_g.append(g_s.numpy()[0, 0]) 

        grad_log.append(g_g)

        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 

        if step % 200 == 0:
            print(f'Epoch {epoch + 1}/{epochs}', end='. ')
            print(f'Step {step}. Train loss: {loss_value}') 

А веса тоже посчитаться не могут, они вышли за разрешенные значения float32.

In [ ]:
model.get_weights()[:2]

Но можем изменить фукнцию активации ReLU.

Раньше у неё не было ограничений в большую сторону, а значит выходные значения активации могли быть очень большими.

Но теперь поставим ограничения в 6, чтобы как-то утихомирить наши сигнала, а значит и веса.

In [ ]:
from keras import backend as K

def my_relu(x):
    return K.relu(x, max_value=6)

In [ ]:
my_relu([10, 0, -2, 7])

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation=my_relu),
    Dense(64, activation=my_relu),
    Dense(32, activation=my_relu),
    Dense(16, activation=my_relu),
    Dense(10, activation='sigmoid')
])


model.summary()

In [ ]:
model.set_weights(new_weights)

In [ ]:
optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=0.01)
loss_fn = CategoricalCrossentropy()

Обучение идёт, nan'ов не видно.

In [ ]:
grad_log = []

epochs = 2 

for epoch in range(epochs): 
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset): 
        
        with tf.GradientTape() as tape: 
            preds = model(x_batch_train) 

            loss_value = loss_fn(y_batch_train, preds)

            grads = tape.gradient(loss_value, model.trainable_weights) 
            g_g = []

            for g_s in grads:
                if len(g_s.numpy().shape) == 2:
                    g_g.append(g_s.numpy()[0, 0]) 

        grad_log.append(g_g)

        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 

        if step % 200 == 0:
            print(f'Epoch {epoch + 1}/{epochs}', end='. ')
            print(f'Step {step}. Train loss: {loss_value}') 

In [ ]:
model.get_weights()[:2]

## Summary

Сегодня обсудили:
1. Какие есть функции активации нейронов (Sigmoid, ReLU, ELU, Leaky ReLU...)
2. Плюсы каждой функции активации
3. Минусы каждой функции активации
4. Как написать свою функцию активации



<table>
<tr>
<th>
Функция активации
</th>

<th>
Плюсы
</th>

<th>
Минусы
</th>
</tr>

<tr>
<td>
linear
</td>

<td>

        1. Меньше времени на вычисления
</td>

<td>

        1. Линейная функция
        2. Нефиксированный диапазон значений функции — [-∞,+∞]
</td>
</tr>


<tr>
<td>
Sigmoid
</td>

<td>

        1. Нелинейная функция
        2. В диапазоне значений сигнала от -2.5 до 2.5 значения активации меняются очень быстро
        3. Фиксированный диапазон значений функции — [0,1]

</td>

<td>

        1. Затухают градиенты 

</td>
</tr>

<tr>
<td>
Tanh
</td>

<td>

        1. Нелинейная функция
        2. В диапазоне значений сигнала от -2.5 до 2.5 значения активации меняются очень быстро
        3. Фиксированный диапазон значений функции — [-1,1]
</td>

<td>

        1. Затухают градиенты 

</td>
</tr>

<tr>
<td>
ReLU
</td>

<td>

        1. Меньше времени на вычисления
        2. Меньше вес у сети
        3. Нет проблемы затухающих градиентов 
    
</td>

<td>

        1. Нулевые градиенты у сигналов, которые меньше 0
        2. Нефиксированный диапазон значений функции
</td>
</tr>

<tr>
<td>
Leaky ReLU
</td>

<td>

        1. Нет проблемы нулевых градиентов

</td>

<td>

        1. Нужно подбирать значение alpha
        2. Нефиксированный диапазон значений функции
</td>
</tr>

<tr>
<td>
ELU
</td>

<td>

        1. Нет проблемы нулевых градиентов
        2. Имеются негативные выходы, что помогает нейронке направлять веса в нужное напрвление
</td>

<td>

        1. Больше времени на вычисления
        2. Нефиксированный диапазон значений функции
        3. Нужно подбирать значение alpha
</td>
</tr>

</table>
